In [ ]:
import sys
from google.colab import drive
# Allow colab to access google drive
drive.mount('/content/drive')
# Add minerl's folder to path, so we can install it with pip
#!chmod 555 -R "/content/drive/MyDrive/packages/minerl"
#sys.path.append("/content/drive/MyDrive/packages/minerl")
#!pip3  install -e "/content/drive/MyDrive/packages/minerl/herobraine/env_specs/stone_collection_specs"

Mounted at /content/drive


In [ ]:
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
!sudo apt-get install xserver-xephyr
!sudo apt install tigervnc-standalone-server
!sudo apt-get install -y python3-opengl
!sudo apt-get install ffmpeg
!pip3 install gym==0.17.3
!pip3 install minerl==0.4.4
#!pip3 install -e /content/drive/MyDrive/packages/minerl
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/openjdk-r/ppa/ubuntu/ jammy main'
More info: https://launchpad.net/~openjdk-r/+archive/ubuntu/ppa
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/openjdk-r-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/openjdk-r-ubuntu-ppa-jammy.list
Traceback (most recent call last):
  File "/usr/bin/add-apt-repository", line 364, in <module>
    sys.exit(0 if addaptrepo.main() else 1)
  File "/usr/bin/add-apt-repository", line 357, in main
    shortcut.add()
  File "/usr/lib/python3/dist-packages/softwareproperties/shortcuthandler.py", line 222, in add
    self.add_key()
  File "/usr/lib/python3/dist-packages/softwareproperties/shortcuthandler.py", line 398, in add_key
    if not all((self.trustedparts_file, self.trustedparts_content)):
  File "/usr/lib/python3/dist-packages/softwareproperties/ppa.py", line 141, in trustedparts_content


In [ ]:
!pip3 install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.3 MB/s eta 0:00:00


In [ ]:
# Copyright (c) 2020 All Rights Reserved
# Author: William H. Guss, Brandon Houghton

from minerl.herobraine.env_specs.simple_embodiment import SimpleEmbodimentEnvSpec
from minerl.herobraine.hero.mc import MS_PER_STEP, STEPS_PER_MS
from minerl.herobraine.hero.handler import Handler
from typing import List

import minerl.herobraine
import minerl.herobraine.hero.handlers as handlers
from minerl.herobraine.env_spec import EnvSpec

# This is the text that would appear in the documentation online if this code were merged with MineRL
STONECOLLECT_DOC = """
In StoneCollection, the agent must collect 32 `minecraft:cobblestone`. This replicates a common scenario
in Minecraft, as cobblestone is necessary to craft a large amount of items in the game and is a
key resource in Minecraft. For example, stone pickaxes and furnaces, which are needed to mine and smelt iron.

The agent begins in a random starting location on a random survival map with a wooden pickaxe.
"""

# Defines how long each episode should last
STONECOLLECT_LENGTH = 8000

# This is the class that implements the environment specification,
# defining things such as the reward function, the starting items for the AI agent, etc.
class StoneCollection(SimpleEmbodimentEnvSpec):

    # Stone collection constructor
    def __init__(self, *args, **kwargs):
        if 'name' not in kwargs:
            kwargs['name'] = 'StoneCollection-v0' # Add environment name if not added

        # End episode if the agent collects 32 cobblestone
        super().__init__(*args,
                         max_episode_steps=STONECOLLECT_LENGTH, reward_threshold=32.0,
                         **kwargs)

    # Produce a reward of 1 for each cobblestone block the agent collects
    def create_rewardables(self) -> List[Handler]:
        return [
            handlers.RewardForCollectingItems([
                dict(type="cobblestone", amount=1, reward=256.0),
                dict(type="dirt", amount=1, reward=64.0),
            ]),
            handlers.RewardForTouchingBlockType([
                dict(type="cobblestone", behaviour="constant", reward=8.0),
                dict(type="dirt", behaviour="constant", reward=1.0),
            ])
        ]

    # Start the agent with a wooden pickaxe in its inventory
    def create_agent_start(self) -> List[Handler]:
        return [
            handlers.SimpleInventoryAgentStart([
                dict(type="wooden_pickaxe", quantity=1)
            ])
        ]

    def create_agent_handlers(self) -> List[Handler]:
        # Again, we will quit the episode if the agent reaches the 32 cobblestone threshold
        return [
            handlers.AgentQuitFromPossessingItem([
                dict(type="cobblestone", amount=32)]
            )
        ]

    def create_server_world_generators(self) -> List[Handler]:
        # We are using the default world generation because we want our agent to be able to mine cobblestone in any environment
        return [handlers.BiomeGenerator("extreme_hills")]

    def create_server_quit_producers(self) -> List[Handler]:
        # Set a timeout to end the episode (to prevent it from running forever if the agent cannot collect 32 cobbelstone)
        return [
            handlers.ServerQuitFromTimeUp(
                (STONECOLLECT_LENGTH * MS_PER_STEP)),
            handlers.ServerQuitWhenAnyAgentFinishes()
        ]

    # This method can be used to change other things about the world such as drawing shapes or spawning a village
    # Not needed for stone collection
    def create_server_decorators(self) -> List[Handler]:
        return []

    # This method sets the conditions for the world the agent will spawn into
    # We will allow spawning and the passage of time to replicate a realistic Minecraft environment
    # and so that the conditions match the closest related dataset, MineRLObtainIronPickaxe-v0
    def create_server_initial_conditions(self) -> List[Handler]:
        return [
            handlers.TimeInitialCondition(
                start_time=6000,
                allow_passage_of_time=True,
            ),
            handlers.SpawningInitialCondition(
                allow_spawning=True
            )
        ]

    # The agent succeeds if its reward surpasses the threshold we set (in our case, 32 cobblestone blocks)
    def determine_success_from_rewards(self, rewards: list) -> bool:
        return sum(rewards) >= self.reward_threshold

    def is_from_folder(self, folder: str) -> bool:
        return folder == 'stonecollection'

    # Returns documentation that would be displayed on the docs website for this environment
    def get_docstring(self):
        return STONECOLLECT_DOC

/usr/local/lib/python3.10/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
#from minerl.herobraine.env_specs.stone_collection_specs import StoneCollection
import logging

In [ ]:
import os
# import gym
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
import stable_baselines3
import matplotlib.pyplot as plt

In [ ]:
#Start the Display for saving videos on Colab
from pyvirtualdisplay import Display
display = Display(visible=False, size=(400, 300))
display.start();

In [ ]:
#A wrapper for getting the POV of the avatar from the environment, which is needed for stable_baselines
class ExtractPOV(gym.ObservationWrapper):
  def __init__(self, env):
    super().__init__(env)
    self.observation_space = self.env.observation_space['pov']

  def observation(self, observation):
    return observation['pov']

In [ ]:
class ReversibleActionWrapper(gym.ActionWrapper):
    def wrap_action(self, inner_action):
        if hasattr(self.env, 'wrap_action'):
            return self.reverse_action(self.env.wrap_action(inner_action))
        else:
            return self.reverse_action(inner_action)

    def reverse_action(self, action):
        raise NotImplementedError("In order to use a ReversibleActionWrapper, you need to implement a `reverse_action` function"
                                  "that is the inverse of the transformation performed on an action that comes into the wrapper")


class ActionShaping(ReversibleActionWrapper):
  def __init__(
      self,
      env: gym.Env,
      camera_angle: int = 10,
      always_attack: bool = True,
      camera_margin: int = 5
  ):

    super().__init__(env)

    self.camera_angle = camera_angle
    self.camera_margin = camera_margin
    self.always_attack = always_attack
    self._actions = [
        [('attack',1)],
        [('forward',1)],
        [('forward',1), ('jump',1)],
        [('camera', [-self.camera_angle,0])],
        [('camera', [self.camera_angle,0])],
        [('camera', [0, self.camera_angle])],
        [('camera', [0, -self.camera_angle])]
    ]

    self.actions = []
    for actions in self._actions:
      act = self.env.action_space.noop()
      for a, v in actions:
        act[a] = v
      if self.always_attack:
        act['attack'] = 1
      self.actions.append(act)

    self.action_space = gym.spaces.Discrete(len(self.actions) + 1)

  def action(self, action):
    if action == 7:
      return self.env.action_space.noop()
    else:
      return self.actions[action]

  def reverse_action(self, action: dict) -> np.ndarray:
    camera_actions = action["camera"].squeeze()
    attack_actions = action["attack"].squeeze()
    forward_actions = action["forward"].squeeze()
    jump_actions = action["jump"].squeeze()
    batch_size = len(camera_actions)
    actions = np.zeros((batch_size,), dtype=int)

    for i in range(len(camera_actions)):
      if camera_actions[i][0] < -self.camera_margin:
        actions[i] = 3
      elif camera_actions[i][0] > self.camera_margin:
        actions[i] = 4
      elif camera_actions[i][1] > self.camera_margin:
        actions[i] = 5
      elif camera_actions[i][1] < -self.camera_margin:
        actions[i] = 6
      elif forward_actions[i] == 1:
        if jump_actions[i] == 1:
          actions[i] = 2
        else:
          actions[i] = 1
      elif attack_actions[i] == 1:
        actions[i] = 0
      else:
        actions[i] = 7

    return actions

In [ ]:
from stable_baselines3.common import results_plotter
from stable_baselines3.common import monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
  def __init__(self, check_freq: int, log_dir: str, verbose=1):
    super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
    self.check_freq = check_freq
    self.log_dir = log_dir
    self.save_path = os.path.join(log_dir, 'best_model')
    self.best_mean_reward = -np.inf

  def _init_callack(self) -> None:
    if self.save_path is not None:
      os.makedirs(self.save_path, exist_ok=True)

  def _on_step(self) -> bool:
    if self.n_calls % self.check_freq == 0:

      #Retrieve  Training Reward
      x, y = ts2xy(load_results(self.log_dir), 'timesteps')
      if len(x) > 0:
          #Mean training reward over the last 100 episodes
          mean_reward = np.mean(y[-100:])
          if self.verbose > 0:
            print("Num timesteps: {}".format(self.num_timesteps))
            print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))

            #New best model, save the agent
            if mean_reward > self.best_mean_reward:
              self.best_mean_reward = mean_reward
              #Example for saving best model
              if self.verbose > 0:
                print("Saving new best model to {}".format(self.save_path))
              self.model.save(self.save_path)
    return True

In [ ]:
abs_STONE = StoneCollection()
abs_STONE.register() # Register with gym

In [ ]:
# import os
# import glob

# # Start from the root directory
# os.chdir('/')

# # Look for the file
# for file in glob.glob('**/launchClient.sh', recursive=True):
#     print(file)


In [ ]:
from stable_baselines3 import PPO

environment = gym.make("StoneCollection-v0")
#environment = gym.make("MineRLTreechop-v0")

In [ ]:
# !sudo chmod -R +x /tmp/tmp*

In [ ]:
# !ls -l /tmp

In [ ]:
#!sudo -E pip3 install gym colabgymrender

In [ ]:
from colabgymrender.recorder import Recorder
env = Recorder(environment, "/content/drive/MyDrive/ml4mc_outputs", fps=60)

ERROR:minerl.env.malmo.instance.fbbf20:[21:46:46] [main/ERROR]: The binary patch set is missing. Either you are in a development environment, or things are not going to work!
ERROR:minerl.env.malmo.instance.fbbf20:[21:46:47] [main/ERROR]: FML appears to be missing any signature data. This is not a good thing
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:23] [Client thread/INFO]: [STDOUT]: [ERROR] Seed specified was NONE. Expected a long (integer).
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:24] [Thread-6/ERROR]: Error in class 'LibraryLWJGLOpenAL'
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:24] [Thread-6/ERROR]: Unable to initialize OpenAL.  Probable cause: OpenAL not supported.
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:24] [Thread-6/WARN]: ERROR MESSAGE:
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:24] [Sound Library Loader/WARN]: ERROR MESSAGE:
ERROR:minerl.env.malmo.instance.fbbf20:[21:47:31] [Thread-10/ERROR]: Error in class 'LibraryLWJGLOpenAL'
ERROR:minerl.env.malmo.inst

In [ ]:
!pip install shimmy

In [ ]:
"""
#callback addition
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)
monitored_env = monitor.Monitor(env1, log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)
"""
obs_wrapped_stone = ExtractPOV(env) #Extracting the POV of the avatar from the environment which is needed for stable_baselines
obs_action_wrapped_stone = ActionShaping(obs_wrapped_stone) #Performing action shaping on the actions of the environment to convert them from dictionaries into an array.
obs = obs_action_wrapped_stone.reset() #reseting the provided environnment



model = PPO(policy="CnnPolicy", env=obs_action_wrapped_stone, verbose=1) #Setting the model to be a PPO model with a CnnPolicy. This was just the model used by tutorials, we'll experiment with the best model later
model.learn(total_timesteps=50000) #Training the model, allowing it to walk through 50000 timesteps of the environment (about 1.5 minutes)
env.release() #releasing the recorded environment to actually make a video on Colab.


ERROR:minerl.env.malmo.instance.fbbf20:[21:49:35] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:49:35] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
  warnings.warn(



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.
ERROR:minerl.env.malmo.instance.fbbf20:[21:52:51] [Client thread/INFO]: [STDOUT]: STATE ERROR - multiple states in the queue.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.33e+03 |
|    ep_rew_mean     | 423      |
| time/              |          |
|    fps             | 9        |
|    iterations      | 1        |
|    time_elapsed    | 208      |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.33e+03    |
|    ep_rew_mean          | 423         |
| time/                   |             |
|    fps                  | 11          |
|    iterations           | 2           |
|    time_elapsed         | 365         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.043151837 |
|    clip_fraction        | 0.373       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.04       |
|    explained_variance   | 0.00302     |
|    learning_rate        | 0.

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.62e+03   |
|    ep_rew_mean          | 4.4e+03    |
| time/                   |            |
|    fps                  | 13         |
|    iterations           | 5          |
|    time_elapsed         | 777        |
|    total_timesteps      | 10240      |
| train/                  |            |
|    approx_kl            | 0.06065808 |
|    clip_fraction        | 0.422      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.79      |
|    explained_variance   | 0.421      |
|    learning_rate        | 0.0003     |
|    loss                 | 27.5       |
|    n_updates            | 40         |
|    policy_gradient_loss | 0.00208    |
|    value_loss           | 138        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.62e+03   |
|    ep_rew_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.72e+03    |
|    ep_rew_mean          | 3.43e+03    |
| time/                   |             |
|    fps                  | 13          |
|    iterations           | 9           |
|    time_elapsed         | 1316        |
|    total_timesteps      | 18432       |
| train/                  |             |
|    approx_kl            | 0.015052928 |
|    clip_fraction        | 0.0262      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.075      |
|    explained_variance   | 0.418       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.78        |
|    n_updates            | 80          |
|    policy_gradient_loss | 0.00188     |
|    value_loss           | 9.57        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 5.72e+03

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 6.28e+03  |
|    ep_rew_mean          | 2.78e+03  |
| time/                   |           |
|    fps                  | 14        |
|    iterations           | 13        |
|    time_elapsed         | 1793      |
|    total_timesteps      | 26624     |
| train/                  |           |
|    approx_kl            | 0.0477857 |
|    clip_fraction        | 0.086     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.218    |
|    explained_variance   | 0.789     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.979     |
|    n_updates            | 120       |
|    policy_gradient_loss | 0.00499   |
|    value_loss           | 2.81      |
---------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6.28e+03   |
|    ep_rew_mean          | 2.78e+03 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.61e+03    |
|    ep_rew_mean          | 2.25e+03    |
| time/                   |             |
|    fps                  | 15          |
|    iterations           | 17          |
|    time_elapsed         | 2284        |
|    total_timesteps      | 34816       |
| train/                  |             |
|    approx_kl            | 0.025808334 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.541      |
|    explained_variance   | 0.908       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0254      |
|    n_updates            | 160         |
|    policy_gradient_loss | -0.00413    |
|    value_loss           | 0.437       |
-----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 6.61e+03  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.84e+03    |
|    ep_rew_mean          | 1.91e+03    |
| time/                   |             |
|    fps                  | 15          |
|    iterations           | 21          |
|    time_elapsed         | 2761        |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.030894117 |
|    clip_fraction        | 0.155       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.24       |
|    explained_variance   | -0.124      |
|    learning_rate        | 0.0003      |
|    loss                 | 8.93        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.00273    |
|    value_loss           | 32.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.84e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7e+03       |
|    ep_rew_mean          | 2.14e+03    |
| time/                   |             |
|    fps                  | 15          |
|    iterations           | 24          |
|    time_elapsed         | 3141        |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.028550426 |
|    clip_fraction        | 0.146       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.566      |
|    explained_variance   | 0.787       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.55        |
|    n_updates            | 230         |
|    policy_gradient_loss | 0.00566     |
|    value_loss           | 7.44        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7e+03 

In [ ]:
"""
results_plotter.plot_results([log_dir], 5000, results_plotter.X_TIMESTEPS, "MineRL RL Training")
plt.show()
"""
model.save("/content/drive/MyDrive/ml4mc_outputs/MineRL_RL_Model_newest", )

In [ ]:
# testEnv = gym.make("MineRLObtainDiamond-v0")
# testRec = Recorder(testEnv, "/content/drive/MyDrive/ml4mc_outputs", fps=60)

# obs_wrapped_test = ExtractPOV(testRec) #Extracting the POV of the avatar from the environment which is needed for stable_baselines
# obs_action_wrapped_test = ActionShaping(obs_wrapped_test) #Performing action shaping on the actions of the environment to convert them from dictionaries into an array.
# obs = obs_action_wrapped_test.reset()

# total_reward = 0
# for episode in range(1):
#   obs = obs_action_wrapped_test.reset() #reseting the provided environnment
#   total_reward = 0
#   for i in range(500):
#     action, _states = model.predict(obs.copy())
#     obs, rewards, done, info = obs_action_wrapped_test.step(action)
#     if done:
#       break
#     total_reward += rewards
#   print(total_reward)


# testRec.release() #releasing the recorded environment to actually make a video on Colab.
